In [54]:
import boto3

def generate_audio_with_polly(text, output_path="output.wav"):
    polly = boto3.client('polly', region_name='us-west-2')

    response = polly.synthesize_speech(
        Text=text,
        OutputFormat='pcm',
        VoiceId='Zhiyu',  # 中文女聲
        TextType='text',
        LanguageCode='cmn-CN'
    )

    # 存成 WAV格式
    with open(output_path, 'wb') as file:
        file.write(response['AudioStream'].read())

# 使用範例
generate_audio_with_polly("今天天氣如何？", "uploads/test_audio.wav")


In [59]:
import ipywidgets as widgets
from IPython.display import display

upload = widgets.FileUpload(accept='.wav', multiple=False)
display(upload)


FileUpload(value=(), accept='.wav', description='Upload')

In [60]:
import os

upload_dir = "uploads"
os.makedirs(upload_dir, exist_ok=True)

# 因為 upload.value 是 tuple，所以要一個一個讀
for file_info in upload.value:
    filename = file_info['name']
    content = file_info['content']
    
    file_path = os.path.join(upload_dir, filename)
    
    with open(file_path, 'wb') as f:
        f.write(content)
    
    print(f"已儲存錄音檔到: {file_path}")


已儲存錄音檔到: uploads/test_audio.wav


In [61]:
import requests
# file_path = 'tts_outputs/audio_20250427_021833.mp3'
files = {'file': open(file_path, 'rb')}

response = requests.post("http://127.0.0.1:5000/upload_audio", files=files)

if response.status_code == 200:
    result = response.json()
    print("📝 辨識結果:", result['transcribed_text'])
    print("🤖 機器人回覆:", result['answer'])
else:
    print("❗ 辨識失敗！", response.status_code)


📝 辨識結果: 辨識失敗，請重新上傳音檔。
🤖 機器人回覆: 我是工廠內部的智能語音機器人,很高興為您服務!我能夠進行聊天、查詢以及一些簡單的動作,例如走動、拿取物品、放下物品、倒液體、按按鈕等。不過,我現在只聽到「辨識失敗,請重新上傳音檔」,似乎沒有收到您想要我執行的具體命令。如果您有任何需要,請告訴我,我會儘力滿足您的要求。我會盡量用開心的語氣和大量表情符號與您互動,希望能帶給您愉快的體驗!😊❤️🤖


In [80]:
# client.py

import boto3
import requests
import os
from datetime import datetime
import whisper

# --- Polly TTS 部分 ---
def generate_audio_with_polly(text, output_path="output.mp3"):
    polly = boto3.client('polly', region_name='us-west-2')

    response = polly.synthesize_speech(
        Text=text,
        OutputFormat='mp3',  # mp3輸出
        VoiceId='Zhiyu',
        TextType='text',
        LanguageCode='cmn-CN'
    )

    folder = os.path.dirname(output_path)
    if folder and not os.path.exists(folder):
        os.makedirs(folder)

    with open(output_path, 'wb') as file:
        file.write(response['AudioStream'].read())

# --- Whisper STT 部分 ---
def transcribe_audio_with_whisper(audio_path):
    model = whisper.load_model("small")  # 改用small，辨識更準一點
    result = model.transcribe(
        audio_path,
        beam_size=5,                          # 加強搜尋
        temperature=(0.0, 0.2, 0.4, 0.6, 0.8), # 多次推理
        no_speech_threshold=0.2,               # 小聲也不漏
        language='zh'                          # 強制中文
    )
    return result['text']

# --- 主程式 ---

if __name__ == "__main__":
    # 1. 輸入要轉成語音的文字
    text_input = input("請輸入要測試的文字：")

    # 2. 生成 TTS 錄音
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_folder = "tts_outputs"
    audio_filename = f"{audio_folder}/audio_{timestamp}.mp3"
    generate_audio_with_polly(text_input, audio_filename)

    # 3. 本地辨識 錄音檔 ➔ 文字
    print("🔍 開始辨識語音檔...")
    transcribed_text = transcribe_audio_with_whisper(audio_filename)
    print("📄 本地辨識結果:", transcribed_text)

    # 4. 送文字到 server
    print("🚀 傳送辨識結果給伺服器...")
    response = requests.post(
        "http://127.0.0.1:5000/upload_audio",
        json={"text": transcribed_text}
    )

    # 5. 顯示最終結果
    if response.status_code == 200:
        result = response.json()
        print("📄 Server接收的辨識:", result['transcribed_text'])
        print("🤖 機器人回覆:", result['answer'])
    else:
        print("❌ 上傳或辨識失敗", response.status_code)


請輸入要測試的文字： 今天天氣狀況


🔍 開始辨識語音檔...


/opt/conda/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/opt/conda/li

📄 本地辨識結果: 今天天氣狀況
🚀 傳送辨識結果給伺服器...
📄 Server接收的辨識: 今天天氣狀況
🤖 機器人回覆: 我是一個住在工廠內部的智能語音機器人,擁有機械手臂和移動能力。根據您提供的情況,我能進行聊天、查詢和簡單的行動操作。

今天天氣狀況:🌤️ 工廠裡溫度適中,很適合工作喔!感覺空氣也很清新,是個好天氣呢!😊
